In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.models as models
import collections
import math
import sys
import numpy as np
sys.path.append("/mnt/batch/tasks/shared/LS_root/mounts/clusters/pyfuse/code/pyramid-fuse")
# # sys.setrecursionlimit(10000000)
import Utils
from Utils.CubePad import CustomPad
image_path = 'Sample.jpg'
# import torchvision.models as models
# from pspnet import PSPNet

In [6]:
import numpy as np

In [7]:
from PIL import Image
import cv2

In [8]:
img = np.asarray(Image.open(image_path), dtype=np.float32)
img=img.transpose(2,0,1)
img=np.expand_dims(img,0)
img=torch.tensor(img)

In [9]:
class CETransform(nn.Module):
    def __init__(self):
        super(CETransform, self).__init__()
        equ_h = [512, 128, 64, 32, 16]
        cube_h = [256, 64, 32, 16, 8]

        self.c2e = dict()
        self.e2c = dict()

        for h in equ_h:
            a = Utils.Equirec2Cube(1, h, h*2, h//2, 90)
            self.e2c['(%d,%d)' % (h, h*2)] = a

        for h in cube_h:
            a = Utils.Cube2Equirec(1, h, h*2, h*4)
            self.c2e['(%d)' % (h)] = a

    def E2C(self, x):
        print(x.shape)
        [bs, c, h, w] = x.shape
        key = '(%d,%d)' % (h, w)
        print(key)
        assert key in self.e2c
        return self.e2c[key].ToCubeTensor(x)

    def C2E(self, x):
        print(x.shape)
        [bs, c, h, w] = x.shape
        key = '(%d)' % (h)
        assert key in self.c2e and h == w
        return self.c2e[key].ToEquirecTensor(x)

    def forward(self, equi, cube):
        return self.e2c(equi), self.c2e(cube)


In [10]:
mypad = getattr(Utils.CubePad, 'SpherePad')

In [11]:
ce = CETransform()

In [12]:
img1 = torch.randn([12, 21, 256, 256]).cuda()

In [13]:
test1 = ce.C2E(img1)

torch.Size([12, 21, 256, 256])


/anaconda/envs/azureml_py36/lib/python3.6/site-packages/torch/nn/functional.py:2705: UserWarning: Default grid_sample and affine_grid behavior has changed to align_corners=False since 1.3.0. Please specify align_corners=True if the old behavior is desired. See the documentation of grid_sample for details.
  warnings.warn("Default grid_sample and affine_grid behavior has changed "


In [14]:
test1.shape

torch.Size([2, 21, 512, 1024])

In [15]:
img2 = torch.randn([6, 21, 64, 64]).cuda()

In [16]:
test2 = ce.C2E(img2)

torch.Size([6, 21, 64, 64])


In [17]:
e = torch.randn([1, 3, 512, 1024])
e.shape

torch.Size([1, 3, 512, 1024])

In [19]:
class PSPModule(nn.Module):
    def __init__(self, features, out_features=1024, sizes=(1, 2, 3, 6)):
        super().__init__()
        self.stages = []
        self.stages = nn.ModuleList([self._make_stage(features, size) for size in sizes])
        self.bottleneck = nn.Conv2d(features * (len(sizes) + 1), out_features, kernel_size=1)
        self.relu = nn.ReLU()

    def _make_stage(self, features, size):
        prior = nn.AdaptiveAvgPool2d(output_size=(size, size))
        conv = nn.Conv2d(features, features, kernel_size=1, bias=False)
        return nn.Sequential(prior, conv)

    def forward(self, feats):
        h, w = feats.size(2), feats.size(3)
        priors = [F.upsample(input=stage(feats), size=(h, w), mode='bilinear') for stage in self.stages] + [feats]
        bottle = self.bottleneck(torch.cat(priors, 1))
        return self.relu(bottle)

In [20]:
class PSPUpsample(nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, 3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.PReLU()
        )

    def forward(self, x):
        h, w = 512, 1024
        if x.shape[0]%6==0:
            h,w = 256, 256
        p = F.upsample(input=x, size=(h, w), mode='bilinear')
        return self.conv(p)

In [21]:
class PSPNet(nn.Module):
    def __init__(self, n_classes=21, sizes=(1, 2, 3, 6), psp_size=1024, deep_features_size=1024):
        super().__init__()
        self.psp = PSPModule(psp_size, 1024, sizes)
        self.drop_1 = nn.Dropout2d(p=0.3)

        self.up_1 = PSPUpsample(1024, 256)
        self.up_2 = PSPUpsample(256, 64)
        self.up_3 = PSPUpsample(64, 64)

        self.drop_2 = nn.Dropout2d(p=0.15)
        self.final = nn.Sequential(
            nn.Conv2d(64, n_classes, kernel_size=1),
            nn.LogSoftmax()
        )

        self.classifier = nn.Sequential(
            nn.Linear(deep_features_size, 256),
            nn.ReLU(),
            nn.Linear(256, n_classes)
        )

    def forward(self, x):
#         print(x.shape)
        f = x
        class_f = x
        p = self.psp(f)
        p = self.drop_1(p)

        p = self.up_1(p)
        p = self.drop_2(p)

        p = self.up_2(p)
        p = self.drop_2(p)

        p = self.up_3(p)
        p = self.drop_2(p)

        auxiliary = F.adaptive_max_pool2d(input=class_f, output_size=(1, 1)).view(-1, class_f.size(1))
   
        return self.final(p), self.classifier(auxiliary)

In [22]:
img1 = torch.randn(1,1024,16,32)

In [23]:
psp = PSPNet()

In [24]:
output = psp(img1)

/anaconda/envs/azureml_py36/lib/python3.6/site-packages/torch/nn/functional.py:2416: UserWarning: nn.functional.upsample is deprecated. Use nn.functional.interpolate instead.
  warnings.warn("nn.functional.upsample is deprecated. Use nn.functional.interpolate instead.")
/anaconda/envs/azureml_py36/lib/python3.6/site-packages/torch/nn/functional.py:2506: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode))
/anaconda/envs/azureml_py36/lib/python3.6/site-packages/torch/nn/modules/container.py:100: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


In [25]:
len(output)

2

In [26]:
output[0].shape

torch.Size([1, 21, 512, 1024])

In [27]:
img2 = torch.randn(6,1024,8,8)


In [28]:
output2 = psp(img2)


In [30]:
output2[0] =

torch.Size([6, 21, 256, 256])

In [31]:
test1 = ce.C2E(output2[0])

torch.Size([6, 21, 256, 256])


RuntimeError: grid_sampler(): expected input and grid to be on same device, but input is on cpu and grid is on cuda:0